# Bandgap Workflow Example

This notebook demonstrates how to build and run a bandgap calculation workflow for materials using the Mat3ra API. The example uses Silicon as the target material and shows how to configure DFT calculations with custom model parameters and k-grids.

## Process Overview
1. Set up environment and parameters
2. Authenticate and initialize API client
3. Create and save material to the platform
4. Create workflow configuration with relaxation and band structure calculations
5. Configure compute resources
6. Create and submit job
7. Monitor job execution
8. Retrieve and display bandgap results

## 1. Set up the environment and parameters

In [ ]:
from datetime import datetime

# Material parameters
FOLDER = "../uploads"
MATERIAL_NAME = "Silicon"

# Workflow parameters
WORKFLOW_SEARCH_TERM = "band_gap.json"
MY_WORKFLOW_NAME = "Band Gap Calculation Workflow"
ADD_RELAXATION = True  # Whether to add relaxation subworkflow before band structure calculation

# Model parameters
MODEL_TYPE = "dft"
MODEL_SUBTYPE = "lda"  # or "gga"
MODEL_FUNCTIONAL = "pz"  # for lda: pz, for gga: pbe

# K-grid parameters
RELAXATION_KGRID = [1, 2, 3]  # k-grid for relaxation
SCF_KGRID = [1, 1, 1]  # k-grid for SCF calculation
NSCF_KGRID = [1, 1, 1]  # k-grid for NSCF calculation

# Job parameters
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
JOB_NAME = f"Band Gap {timestamp}"

POLL_INTERVAL = 30 # seconds

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install("mat3ra-api-examples", deps=False)
    await micropip.install("mat3ra-utils")
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("api_examples")

## 2. Authenticate and initialize API client
### 2.1. Upon running of this cell, the browser with Mat3ra CLI login page will open.

In [ ]:
from utils.auth import authenticate

# Authenticate and have credentials stored in environment variables
await authenticate()

### 2.2. Initialize API client
Authorization is done via environment variables, where token is stored after authentication.

In [ ]:
from mat3ra.api_client import APIClient

client = APIClient.authenticate()

## 3. Create material
### 3.1. Load material from local file (or Standata)

In [ ]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials
from utils.visualize import visualize_materials as visualize
from utils.jupyterlite import load_material_from_folder

material = load_material_from_folder(FOLDER, MATERIAL_NAME) or Material.create(
    Materials.get_by_name_first_match(MATERIAL_NAME))

visualize(material)

### 3.2. Save material to the platform

In [ ]:
from utils.generic import dict_to_namespace

MY_USER_ID = client.my_account.id
print(f"✅ My ID: {MY_USER_ID}")

material.basis.set_labels_from_list([])
saved_material_response = client.materials.create(material.to_dict(), owner_id=MY_USER_ID)
saved_material = dict_to_namespace(saved_material_response)
print(f"✅ Material created: {saved_material._id}")

### 3.3. Get material id

In [ ]:
print("Material ID:", saved_material._id)

## 5. Create workflow and set its parameters
### 5.1. Get list of applications and select one

In [ ]:
from mat3ra.standata.applications import ApplicationStandata
from mat3ra.ade.application import Application

apps_list = ApplicationStandata.list_all()

In [ ]:
app_config = ApplicationStandata.get_by_name_first_match("espresso")
app = Application(**app_config)
app.name

### 5.2. Create workflow from standard workflows and preview it

In [ ]:
from mat3ra.standata.workflows import WorkflowStandata
from mat3ra.wode.workflows import Workflow
from utils.visualize import visualize_workflow

workflow_config = WorkflowStandata.filter_by_application(app.name).get_by_name_first_match(WORKFLOW_SEARCH_TERM)
workflow = Workflow.create(workflow_config)

visualize_workflow(workflow)

### 5.3. Add relaxation subworkflow

In [ ]:
from utils.visualize import visualize_workflow

if ADD_RELAXATION:
    workflow.add_relaxation()
    # Relaxation subworkflow is added as the first subworkflow
    visualize_workflow(workflow)

### 5.4. Change subworkflow details (Model subtype)
#### 5.4.1. Get available model subtypes and functionals

In [ ]:
from mat3ra.standata.model_tree import ModelTreeStandata

# Change model subtype for relaxation subworkflow
subtypes = ModelTreeStandata.get_subtypes_by_model_type(MODEL_TYPE)
functionals = ModelTreeStandata.get_functionals_by_subtype(MODEL_TYPE, MODEL_SUBTYPE)

#### 5.4.2. Modify model in subworkflow units

In [ ]:
from mat3ra.mode import Model

RELAXATION_SWF_INDEX = 0 if ADD_RELAXATION else None
BAND_GAP_SWF_INDEX = 1 if ADD_RELAXATION else 0

swf_0 = workflow.subworkflows[RELAXATION_SWF_INDEX] if ADD_RELAXATION else None
swf_1 = workflow.subworkflows[BAND_GAP_SWF_INDEX]

model_config = ModelTreeStandata.get_model_by_parameters(
    type=MODEL_TYPE,
    subtype=MODEL_SUBTYPE,
    functional={"slug": MODEL_FUNCTIONAL},
)

method_config = {"type": "pseudopotential", "subtype": "us"}
model_config["method"] = method_config

model = Model.create(model_config)
if ADD_RELAXATION:
    swf_0.model = model
swf_1.model = model
print(model)


### 5.5. Modify k-grid in subworkflow units
#### 5.5.1. Get k-grid context

In [ ]:
from mat3ra.wode.context.providers import PointsGridDataProvider

new_context_relax = PointsGridDataProvider(dimensions=RELAXATION_KGRID, isEdited=True).yield_data() if ADD_RELAXATION else None
new_context_scf = PointsGridDataProvider(dimensions=SCF_KGRID, isEdited=True).yield_data()
new_context_nscf = PointsGridDataProvider(dimensions=NSCF_KGRID, isEdited=True).yield_data()

#### 5.5.3. Modify workflow units with new context

In [ ]:
# Get workflow's specific unit that needs to be modified
if ADD_RELAXATION:
    relaxation_subworkflow = workflow.subworkflows[RELAXATION_SWF_INDEX]
    unit_to_modify_relax = relaxation_subworkflow.get_unit_by_name(name_regex="relax")
    unit_to_modify_relax.add_context(new_context_relax)
    # Set the modified unit back to the workflow
    # Option 1: direct set by unit object, replacing the existing one
    relaxation_subworkflow.set_unit(unit_to_modify_relax)

band_gap_subworkflow = workflow.subworkflows[BAND_GAP_SWF_INDEX]
unit_to_modify_scf = band_gap_subworkflow.get_unit_by_name(name="pw_scf")
unit_to_modify_scf.add_context(new_context_scf)
unit_to_modify_nscf = band_gap_subworkflow.get_unit_by_name(name="pw_nscf")
unit_to_modify_nscf.add_context(new_context_nscf)

# Option 2: set by unit flowchart id and new unit object
band_gap_subworkflow.set_unit(unit_flowchart_id=unit_to_modify_scf.flowchart_id, new_unit=unit_to_modify_scf)
band_gap_subworkflow.set_unit(unit_flowchart_id=unit_to_modify_nscf.flowchart_id, new_unit=unit_to_modify_nscf)
workflow.name = workflow.name + " (custom k-grids)"
visualize_workflow(workflow)

### 5.6. Save workflow to collection

In [ ]:
workflow_dict = workflow.to_dict()

saved_workflow_response = client.workflows.create(workflow_dict, owner_id=MY_USER_ID)

saved_workflow = dict_to_namespace(saved_workflow_response)
print(f"✅ Workflow created: {saved_workflow._id}")

## 6. Create the compute configuration

In [ ]:
from utils.api import get_cluster_name

CLUSTER_NAME = get_cluster_name()

compute = client.jobs.get_compute(
    cluster=CLUSTER_NAME
)

## 7. Create the job with material and workflow configuration
### 7.1. Get default project id

In [ ]:
projects = client.projects.list({"isDefault": True, "owner._id": MY_USER_ID})
project_id = projects[0]["_id"]


### 7.2. Create job

In [ ]:
from utils.visualize import display_JSON

material_from_collection = client.materials.get(saved_material._id)

print(f"📦 Material: {material_from_collection['_id']}")
print(f"📦 Workflow: {saved_workflow._id}")
print(f"📦 Project: {project_id}")

job_response = client.jobs.create_by_ids(
    materials=[vars(saved_material)],
    workflow_id=saved_workflow._id,
    project_id=project_id,
    prefix=JOB_NAME,
    owner_id=MY_USER_ID,
    compute=compute,
)

job_dict = job_response[0]
job = dict_to_namespace(job_dict)

print("✅ Job created successfully!")
display_JSON(job_response)

## 8. Submit the job and monitor the status

In [ ]:
client.jobs.submit(job._id)

In [ ]:
from utils.api import wait_for_jobs_to_finish

wait_for_jobs_to_finish(client.jobs, [job._id], poll_interval=POLL_INTERVAL)

## 9. Retrieve results

In [ ]:
property_endpoints = client.properties

workflow = Workflow.create(job_dict["workflow"])
unit_flowchart_id = workflow.subworkflows[BAND_GAP_SWF_INDEX].get_unit_by_name(name="pw_nscf").flowchart_id
band_gap_direct = property_endpoints.get_direct_band_gap(job._id, unit_flowchart_id)
band_gap_indirect = property_endpoints.get_indirect_band_gap(job._id, unit_flowchart_id)

print(f"Direct band gap: {band_gap_direct:0.3f} eV")
print(f"Indirect band gap: {band_gap_indirect:0.3f} eV")